<a href="https://colab.research.google.com/github/danielhcg/smishing-detection-dl/blob/main/smishing_dl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Colab Shareable Link:
https://colab.research.google.com/drive/1xWgjgZJHjXQ_lOvINxCHiT3bbgP0RDPK?usp=sharing


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install langid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941172 sha256=7f2964e59cc806a444c6bb5436deaf788e21c04abcc47830d931b70879c7fd16
  Stored in directory: /root/.cache/pip/wheels/23/c8/c6/eed80894918490a175677414d40bd7c851413bbe03d4856c3c
Successfully built langid


In [ ]:
# Load the dependencies and your data set.
import keras
from keras.datasets import imdb
# from keras_preprocessing.sequence import pad_sequences
from keras.preprocessing.sequence import pad_sequences
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.layers import Embedding, SpatialDropout1D, Conv1D, GlobalMaxPool1D, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from keras.layers import LSTM, Bidirectional # LSTM
from keras.layers import GRU # GRU
from keras.models import Model # Non sequential model
from keras.layers import Input, concatenate # Non sequential model

from keras.layers import SimpleRNN

import pickle
import numpy as np
from keras.preprocessing.text import Tokenizer
import time

## Evaluating the data

In [ ]:
import pandas as pd

spam_dataset = pd.read_csv('gdrive/MyDrive/CS673 ANN/Project/Dataset/spam.csv')
ham_dataset = pd.read_csv('gdrive/MyDrive/CS673 ANN/Project/Dataset/ham.csv')
smishing_dataset = pd.read_csv('gdrive/MyDrive/CS673 ANN/Project/Dataset/smishing.csv')

# Print the shape of the datasets
print("Spam dataset shape:    ", spam_dataset.shape)
print("Ham dataset shape:     ", ham_dataset.shape)
print("Smishing dataset shape:", smishing_dataset.shape)

FileNotFoundError: [Errno 2] No such file or directory: 'gdrive/MyDrive/CS673 ANN/Project/Dataset/spam.csv'

In [ ]:
# Loading Tweets spam
tweet_2018 = pd.read_csv('gdrive/MyDrive/CS673 ANN/Project/Dataset/2018.csv')
tweet_2019 = pd.read_csv('gdrive/MyDrive/CS673 ANN/Project/Dataset/2019.csv')
tweet_2020 = pd.read_csv('gdrive/MyDrive/CS673 ANN/Project/Dataset/2020.csv')
tweet_2021 = pd.read_csv('gdrive/MyDrive/CS673 ANN/Project/Dataset/2021.csv')
tweet_2022 = pd.read_csv('gdrive/MyDrive/CS673 ANN/Project/Dataset/2022.csv')

print("2018 dataset shape:", tweet_2018.shape)
print("2019 dataset shape:", tweet_2019.shape)
print("2020 dataset shape:", tweet_2020.shape)
print("2021 dataset shape:", tweet_2021.shape)
print("2022 dataset shape:", tweet_2022.shape)

In [ ]:
# Adding new data of smishing
new_smishing_dataset = pd.read_csv('gdrive/MyDrive/CS673 ANN/Project/smishing new dataset.csv')

print("New Smishing dataset shape:", new_smishing_dataset.shape)

In [ ]:
# Concatenating smishing
smishing_concat = pd.concat([smishing_dataset, new_smishing_dataset], axis=0)
smishing_concat.shape

In [ ]:
# Concatenating tweets
tweet_concat = pd.concat([tweet_2018, tweet_2019, tweet_2020, tweet_2021, tweet_2022], axis=0)
tweet_concat.shape

In [ ]:
# Total Spam concat
spam = pd.concat([tweet_concat, spam_dataset], axis=0)
spam.shape

## Filtering the data

In [ ]:
# Removing duplicates of smish
smish_no_duplicates = smishing_concat.drop_duplicates()
smish_no_duplicates.shape

In [ ]:
# Removing duplicates of spam
spam_no_duplicates = spam.drop_duplicates()
spam_no_duplicates.shape

In [ ]:
spam_no_duplicates.head()

In [ ]:
import langid
import pandas as pd

# Function to identify the language of a text
def identify_language(text):
    try:
        lang, _ = langid.classify(text)
        return lang
    except Exception as e:
        return None

# Add a new column 'language' to the DataFrame
spam_no_duplicates['language'] = spam_no_duplicates['Text'].apply(identify_language)

# Filter out non-English statements
spam_no_duplicates_english = spam_no_duplicates[spam_no_duplicates['language'] == 'en']

# Display the DataFrame with only English statements
print(spam_no_duplicates_english.shape,'\nDropping extra column')
spam_final_dataset = spam_no_duplicates_english.drop('language', axis=1)
spam_final_dataset.shape

In [ ]:
# Removing duplicates of ham
ham_no_duplicates = ham_dataset.drop_duplicates()
ham_no_duplicates.shape

## Splitting the dataset

In [ ]:
from sklearn.model_selection import train_test_split

# Combine all datasets into one DataFrame
all_data = pd.concat([spam_final_dataset, ham_no_duplicates, smish_no_duplicates], ignore_index=True)

# Combine similar labels and replace 'ham' with 'Ham'
all_data['Label'] = all_data['Label'].replace({'spam': 'Spam', 'smishing': 'Smishing', 'ham': 'Ham'})

X = all_data['Text']
y = all_data['Label']

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Display the shapes of the resulting datasets
print("X_train shape:", x_train.shape)
print("X_test shape:", x_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

In [ ]:
x_train[400:410]

In [ ]:
y_train[400:410]

In [ ]:
y_train.value_counts()

In [ ]:
label_counts = y_train.value_counts()
label_counts.plot(kind='bar')
plt.title('Distribution of Labels in y_train')
plt.xlabel('Label')
plt.ylabel('Count')
plt.show()

In [ ]:
y_test.value_counts()

In [ ]:
label_counts = y_test.value_counts()
label_counts.plot(kind='bar')
plt.title('Distribution of Labels in y_val')
plt.xlabel('Label')
plt.ylabel('Count')
plt.show()

## Hyper Parameters setting


In [ ]:
# 4
#    Set your hyperparameters:

#    epochs,
#    batch size,
#    dimension,
#    unique words,
#    stop words,
#    maximum message length,
#    padding type,
#    truncation type,
#    #dense layer,
#    dropout,
#    n_filters,
#    optimizers,
#    or anything else needed.

num_classes= 3

output_dir = 'model_output/dense'
epochs = 7
batch_size = 128

n_dim = 100
n_unique_words = 5000
total_elements = 5405
n_words_to_skip = 50
pad_type = trunc_type = 'pre'
n_dense = 100
dropout = 0.5

#CNN
cnn_max_review =100
cnn_dense = 256
cnn_dropout = 0.2
n_conv = 256
k_conv = 3

#RNN
n_rnn = 256
drop_rnn = 0.2
drop_embed = 0.2

#LSTM
out_put_ = 'model_output/conv'
lstm_max_review = 100
lstm_drop_embed = 0.2
lstm_n_conv = 256
lstm_k_conv = 0.2
lstm_dropout = 0.2

In [ ]:
# LSTM layer architecture:
n_lstm = 256
drop_lstm = 0.2

# CNN architecture
n_conv = 64
k_conv = 3
mp_size=4

# GRU architecture
n_gru = 256
drop_gru = 0.2

# LSTM stack layer architecture
n_lstm_1 = 64
n_lstm_2 = 64
drop_lstm = 0.2

# Multi conv (Non-sequential)
n_conv_1 = n_conv_2 = n_conv_3 = 256
k_conv_1 = 3
k_conv_2 = 2
k_conv_3 = 4

## Preprocessing


In [ ]:
# 3
# Apply all ( or subset of) the preprocessing steps that we learned in the class:
# Tokenization
# Convertion to lowercase
# Removing stop words
# Removing puntuation
# Stemming
# n-grams

max_message_length = 100

from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()
vect.fit(x_train)

# Tokenizing
token = Tokenizer(num_words=n_unique_words)
token.fit_on_texts(x_train)

x_train_emb = token.texts_to_sequences(x_train)
x_test_emb = token.texts_to_sequences(x_test)

print(x_train[0])
print(x_train_emb[0])

x_train = pad_sequences(x_train_emb, padding='pre', maxlen=max_message_length)
x_test = pad_sequences(x_test_emb, padding='pre', maxlen=max_message_length)

print(x_train[1])

In [ ]:
y_train_original = y_train
y_test_original = y_test

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

## Model 1 Dense

In [ ]:
from keras.callbacks import ModelCheckpoint

# Output Directory name:
output_dir = 'model_output/dense' # Store model's parameter after each epoch, useful at a later time

# If the output_dir directory doesn't exist, we use the mkdirs() method to make it
if not os.path.exists(output_dir):
  os.makedirs(output_dir)

# Allow to save our model parameters after each epoch during training
modelcheckpoint = ModelCheckpoint(filepath = output_dir + '/weights.{epoch:02d}.hdf5')

In [ ]:
from tensorflow.keras.layers import Embedding, SpatialDropout1D, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Dense
model1 = Sequential()
model1.add(Embedding(n_unique_words, n_dim, input_length=max_message_length))
model1.add(Flatten())
model1.add(Dense(n_dense, activation='relu'))
model1.add(Dropout(dropout))
model1.add(Dense(n_dense, activation='relu'))
model1.add(Dropout(dropout))
model1.add(Dense(num_classes, activation='softmax'))  # Change to num_classes

# Compile
model1.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model1.summary()

start_time = time.time()
history1 = model1.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test), callbacks=[modelcheckpoint])
print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed

### Evaluate

In [ ]:
plt.figure(figsize=(10, 4))

# Visualizing training and validation accuracy for this Model
plt.subplot(1, 2, 1)
plt.plot(history1.history['accuracy'])
plt.plot(history1.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Visualizing training and validation loss for this Model
plt.subplot(1, 2, 2)
plt.plot(history1.history['loss'])
plt.plot(history1.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss/Error')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np

# Evaluating the model
y_pred_prob = model1.predict(x_test)
y_pred = y_pred_prob.argmax(axis=-1)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {100*accuracy:.2f}")

In [ ]:
# Plot ROC curve for model
plt.figure(figsize=(8, 6))
plt.hist(y_pred_prob)
_ = plt.axvline(x=0.5, color='orange')
plt.title("Model ROC-AUC")
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

# Assuming y_test_encoded and y_pred_prob are defined
# y_test_encoded: True labels (encoded, not one-hot encoded)
# y_pred_prob: Predicted probabilities for each class

# Calculate ROC-AUC score for model (one-vs-all approach for multi-class)
roc_auc = []

# Create a figure and axis
plt.figure(figsize=(8, 6))

# Plot ROC curve for each class
for i in range(num_classes):
    fpr, tpr, _ = roc_curve(y_test== i, y_pred_prob[:, i])
    roc_auc_class = auc(fpr, tpr)
    roc_auc.append(roc_auc_class * 100.0)
    plt.plot(fpr, tpr, label=f'Class {i} (AUC = {roc_auc_class:.2f})')

# Set plot title and labels
plt.title("Model ROC Curves for Multi-class Classification")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

## Model 2 CNN

In [ ]:
# Output Directory name:
output_dir = 'model_output/CNN' # Store model's parameter after each epoch, useful at a later time

# Allow to save our model parameters after each epoch during training
modelcheckpoint = ModelCheckpoint(filepath = output_dir + '/weights.{epoch:02d}.hdf5')

# If the output_dir directory doesn't exist, we use the mkdirs() method to make it
if not os.path.exists(output_dir):
  os.makedirs(output_dir)

In [ ]:
# CNN
model2 = Sequential()
model2.add(Embedding(n_unique_words, n_dim, input_length=cnn_max_review))
model2.add(SpatialDropout1D(drop_embed))
model2.add(Conv1D(n_conv, k_conv, activation='relu'))
model2.add(GlobalMaxPool1D())
model2.add(Dense(cnn_dense, activation='relu'))
model2.add(Dropout(cnn_dropout))
model2.add(Dense(num_classes, activation='softmax'))  # Change to num_classes
model2.summary()

model2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

start_time = time.time()
history2 = model2.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test), callbacks=[modelcheckpoint])
print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed

### Evaluate

In [ ]:
plt.figure(figsize=(10, 4))

# Visualizing training and validation accuracy for this Model
plt.subplot(1, 2, 1)
plt.plot(history2.history['accuracy'])
plt.plot(history2.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Visualizing training and validation loss for this Model
plt.subplot(1, 2, 2)
plt.plot(history2.history['loss'])
plt.plot(history2.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss/Error')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np

# Evaluating the model
y_pred_prob = model2.predict(x_test)
y_pred = y_pred_prob.argmax(axis=-1)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {100*accuracy:.2f}")

In [ ]:
# Plot ROC curve for model
plt.figure(figsize=(8, 6))
plt.hist(y_pred_prob)
_ = plt.axvline(x=0.5, color='orange')
plt.title("Model ROC-AUC")
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

# Assuming y_test_encoded and y_pred_prob are defined
# y_test_encoded: True labels (encoded, not one-hot encoded)
# y_pred_prob: Predicted probabilities for each class

# Calculate ROC-AUC score for model (one-vs-all approach for multi-class)
roc_auc = []

# Create a figure and axis
plt.figure(figsize=(8, 6))

# Plot ROC curve for each class
for i in range(num_classes):
    fpr, tpr, _ = roc_curve(y_test== i, y_pred_prob[:, i])
    roc_auc_class = auc(fpr, tpr)
    roc_auc.append(roc_auc_class * 100.0)
    plt.plot(fpr, tpr, label=f'Class {i} (AUC = {roc_auc_class:.2f})')

# Set plot title and labels
plt.title("Model ROC Curves for Multi-class Classification")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

## Model 3 RNN

In [ ]:
# Output Directory name:
output_dir = 'model_output/Simple RNN' # Store model's parameter after each epoch, useful at a later time

# If the output_dir directory doesn't exist, we use the mkdirs() method to make it
if not os.path.exists(output_dir):
  os.makedirs(output_dir)

# Allow to save our model parameters after each epoch during training
modelcheckpoint = ModelCheckpoint(filepath = output_dir + '/weights.{epoch:02d}.hdf5')

drop_embed = 0.2
n_rnn = 256
drop_rnn = 0.2
#n_dense = 256
#dropout = 0.2

In [ ]:
# RNN
model3 = Sequential()
model3.add(Embedding(n_unique_words, n_dim, input_length=max_message_length))
model3.add(SpatialDropout1D(drop_embed))
model3.add(SimpleRNN(n_rnn, dropout=drop_rnn))
model3.add(Dense(num_classes, activation='softmax'))  # Change to num_classes

model3.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model3.summary()

start_time = time.time()
history3 = model3.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test), callbacks=[modelcheckpoint])
print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed

### Evaluate

In [ ]:
plt.figure(figsize=(10, 4))

# Visualizing training and validation accuracy for this Model
plt.subplot(1, 2, 1)
plt.plot(history3.history['accuracy'])
plt.plot(history3.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Visualizing training and validation loss for this Model
plt.subplot(1, 2, 2)
plt.plot(history3.history['loss'])
plt.plot(history3.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss/Error')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np

# Evaluating the model
y_pred_prob = model3.predict(x_test)
y_pred = y_pred_prob.argmax(axis=-1)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {100*accuracy:.2f}")

In [ ]:
# Plot ROC curve for model
plt.figure(figsize=(8, 6))
plt.hist(y_pred_prob)
_ = plt.axvline(x=0.5, color='orange')
plt.title("Model ROC-AUC")
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

# Assuming y_test_encoded and y_pred_prob are defined
# y_test_encoded: True labels (encoded, not one-hot encoded)
# y_pred_prob: Predicted probabilities for each class

# Calculate ROC-AUC score for model (one-vs-all approach for multi-class)
roc_auc = []

# Create a figure and axis
plt.figure(figsize=(8, 6))

# Plot ROC curve for each class
for i in range(num_classes):
    fpr, tpr, _ = roc_curve(y_test== i, y_pred_prob[:, i])
    roc_auc_class = auc(fpr, tpr)
    roc_auc.append(roc_auc_class * 100.0)
    plt.plot(fpr, tpr, label=f'Class {i} (AUC = {roc_auc_class:.2f})')

# Set plot title and labels
plt.title("Model ROC Curves for Multi-class Classification")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

## Model 4 LSTM

In [ ]:
# Output Directory name:
output_dir = 'model_output/LSTM' # Store model's parameter after each epoch, useful at a later time

# If the output_dir directory doesn't exist, we use the mkdirs() method to make it
if not os.path.exists(output_dir):
  os.makedirs(output_dir)

# Allow to save our model parameters after each epoch during training
modelcheckpoint = ModelCheckpoint(filepath = output_dir + '/weights.{epoch:02d}.hdf5')


In [ ]:
# LSTM
model4 = Sequential()
model4.add(Embedding(n_unique_words, n_dim, input_length=lstm_max_review))
model4.add(SpatialDropout1D(lstm_drop_embed))
model4.add(LSTM(n_lstm, dropout=lstm_dropout))
model4.add(Dense(num_classes, activation='softmax'))  # Change to num_classes

model4.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model4.summary()

start_time = time.time()
history4 = model4.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test), callbacks=[modelcheckpoint])
print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed

### Evaluate

In [ ]:
plt.figure(figsize=(10, 4))

# Visualizing training and validation accuracy for this Model
plt.subplot(1, 2, 1)
plt.plot(history4.history['accuracy'])
plt.plot(history4.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Visualizing training and validation loss for this Model
plt.subplot(1, 2, 2)
plt.plot(history4.history['loss'])
plt.plot(history4.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss/Error')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np
# Output Directory name:
output_dir = 'model_output/LSTM'
# Evaluating the model
model4.load_weights(output_dir + "/weights.04.hdf5")
y_pred_prob = model4.predict(x_test)
y_pred = y_pred_prob.argmax(axis=-1)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {100*accuracy:.2f}")

In [ ]:
# Plot ROC curve for model
plt.figure(figsize=(8, 6))
plt.hist(y_pred_prob)
_ = plt.axvline(x=0.5, color='orange')
plt.title("Model ROC-AUC")
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

# Assuming y_test_encoded and y_pred_prob are defined
# y_test_encoded: True labels (encoded, not one-hot encoded)
# y_pred_prob: Predicted probabilities for each class

# Calculate ROC-AUC score for model (one-vs-all approach for multi-class)
roc_auc = []

# Create a figure and axis
plt.figure(figsize=(8, 6))

# Plot ROC curve for each class
for i in range(num_classes):
    fpr, tpr, _ = roc_curve(y_test== i, y_pred_prob[:, i])
    roc_auc_class = auc(fpr, tpr)
    roc_auc.append(roc_auc_class * 100.0)
    plt.plot(fpr, tpr, label=f'Class {i} (AUC = {roc_auc_class:.2f})')

# Set plot title and labels
plt.title("Model ROC Curves for Multi-class Classification")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

## Model 5 Improved CNN

In [ ]:
# Output Directory name:
output_dir = 'model_output/Improved-CNN' # Store model's parameter after each epoch, useful at a later time

# If the output_dir directory doesn't exist, we use the mkdirs() method to make it
if not os.path.exists(output_dir):
  os.makedirs(output_dir)

# Allow to save our model parameters after each epoch during training
modelcheckpoint = ModelCheckpoint(filepath = output_dir + '/weights.{epoch:02d}.hdf5')

In [ ]:
#    Tune your model
# Select the model which performed the best in the previous step and then tune the hyperparameters
# Improved CNN
model5 = Sequential()

model5.add(Embedding(n_unique_words, n_dim, input_length=cnn_max_review))
model5.add(SpatialDropout1D(drop_embed))

model5.add(Conv1D(n_conv,k_conv,activation = 'relu'))
model5.add(GlobalMaxPool1D())

model5.add(Dense(64, activation='relu'))
model5.add(Dense(128,activation='relu'))
model5.add(Dropout(0.4))
model5.add(Dense(num_classes, activation='softmax'))

model5.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model5.summary()

start_time = time.time()
history5 =  model5.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test), callbacks=[modelcheckpoint])
print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed

### Evaluate

In [ ]:
plt.figure(figsize=(10, 4))

# Visualizing training and validation accuracy for this Model
plt.subplot(1, 2, 1)
plt.plot(history5.history['accuracy'])
plt.plot(history5.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Visualizing training and validation loss for this Model
plt.subplot(1, 2, 2)
plt.plot(history5.history['loss'])
plt.plot(history5.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss/Error')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np

# Evaluating the model
y_pred_prob = model5.predict(x_test)
y_pred = y_pred_prob.argmax(axis=-1)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {100*accuracy:.2f}")

In [ ]:
# Plot ROC curve for model
plt.figure(figsize=(8, 6))
plt.hist(y_pred_prob)
_ = plt.axvline(x=0.5, color='orange')
plt.title("Model ROC-AUC")
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

# Assuming y_test_encoded and y_pred_prob are defined
# y_test_encoded: True labels (encoded, not one-hot encoded)
# y_pred_prob: Predicted probabilities for each class

# Calculate ROC-AUC score for model (one-vs-all approach for multi-class)
roc_auc = []

# Create a figure and axis
plt.figure(figsize=(8, 6))

# Plot ROC curve for each class
for i in range(num_classes):
    fpr, tpr, _ = roc_curve(y_test== i, y_pred_prob[:, i])
    roc_auc_class = auc(fpr, tpr)
    roc_auc.append(roc_auc_class * 100.0)
    plt.plot(fpr, tpr, label=f'Class {i} (AUC = {roc_auc_class:.2f})')

# Set plot title and labels
plt.title("Model ROC Curves for Multi-class Classification")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

## Model 6 GRU

In [ ]:
# Output Directory name:
output_dir = 'model_output/GRU' # Store model's parameter after each epoch, useful at a later time

# If the output_dir directory doesn't exist, we use the mkdirs() method to make it
if not os.path.exists(output_dir):
  os.makedirs(output_dir)

# Allow to save our model parameters after each epoch during training
modelcheckpoint = ModelCheckpoint(filepath = output_dir + '/weights.{epoch:02d}.hdf5')

In [ ]:
# GRU
model6 = Sequential()
model6.add(Embedding(n_unique_words, n_dim, input_length=max_message_length))
model6.add(SpatialDropout1D(drop_embed))
model6.add(GRU(n_gru, dropout=drop_gru))
model6.add(Dense(num_classes, activation='softmax'))

model6.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model6.summary()

start_time = time.time()
historyGRU =  model6.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test), callbacks=[modelcheckpoint])
print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed

### Evaluate

In [ ]:
plt.figure(figsize=(10, 4))

# Visualizing training and validation accuracy for this Model
plt.subplot(1, 2, 1)
plt.plot(historyGRU.history['accuracy'])
plt.plot(historyGRU.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Visualizing training and validation loss for this Model
plt.subplot(1, 2, 2)
plt.plot(historyGRU.history['loss'])
plt.plot(historyGRU.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss/Error')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np

# Evaluating the model
y_pred_prob = model6.predict(x_test)
y_pred = y_pred_prob.argmax(axis=-1)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {100*accuracy:.2f}")

In [ ]:
# Plot ROC curve for model
plt.figure(figsize=(8, 6))
plt.hist(y_pred_prob)
_ = plt.axvline(x=0.5, color='orange')
plt.title("Model ROC-AUC")
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

# Assuming y_test_encoded and y_pred_prob are defined
# y_test_encoded: True labels (encoded, not one-hot encoded)
# y_pred_prob: Predicted probabilities for each class

# Calculate ROC-AUC score for model (one-vs-all approach for multi-class)
roc_auc = []

# Create a figure and axis
plt.figure(figsize=(8, 6))

# Plot ROC curve for each class
for i in range(num_classes):
    fpr, tpr, _ = roc_curve(y_test== i, y_pred_prob[:, i])
    roc_auc_class = auc(fpr, tpr)
    roc_auc.append(roc_auc_class * 100.0)
    plt.plot(fpr, tpr, label=f'Class {i} (AUC = {roc_auc_class:.2f})')

# Set plot title and labels
plt.title("Model ROC Curves for Multi-class Classification")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

## Model 7 Bi-LSTM

In [ ]:
# Output Directory name:
output_dir = 'model_output/Bi-LSTM' # Store model's parameter after each epoch, useful at a later time

# If the output_dir directory doesn't exist, we use the mkdirs() method to make it
if not os.path.exists(output_dir):
  os.makedirs(output_dir)

# Allow to save our model parameters after each epoch during training
modelcheckpoint = ModelCheckpoint(filepath = output_dir + '/weights.{epoch:02d}.hdf5')

In [ ]:
# 7
model7 = Sequential()
model7.add(Embedding(n_unique_words, n_dim, input_length=max_message_length))
model7.add(SpatialDropout1D(drop_embed))
model7.add(Bidirectional(LSTM(n_lstm_1, dropout=0.4, return_sequences=True)))
model7.add(Bidirectional(LSTM(n_lstm_2, dropout=0.4)))
model7.add(Dense(num_classes, activation='softmax'))

model7.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model7.summary()

start_time = time.time()
history7 =  model7.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test), callbacks=[modelcheckpoint])
print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed

### Evaluate

In [ ]:
plt.figure(figsize=(10, 4))

# Visualizing training and validation accuracy for this Model
plt.subplot(1, 2, 1)
plt.plot(history7.history['accuracy'])
plt.plot(history7.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Visualizing training and validation loss for this Model
plt.subplot(1, 2, 2)
plt.plot(history7.history['loss'])
plt.plot(history7.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss/Error')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np

# Evaluating the model
y_pred_prob = model7.predict(x_test)
y_pred = y_pred_prob.argmax(axis=-1)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {100*accuracy:.2f}")

In [ ]:
# Plot ROC curve for model
plt.figure(figsize=(8, 6))
plt.hist(y_pred_prob)
_ = plt.axvline(x=0.5, color='orange')
plt.title("Model ROC-AUC")
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

# Assuming y_test_encoded and y_pred_prob are defined
# y_test_encoded: True labels (encoded, not one-hot encoded)
# y_pred_prob: Predicted probabilities for each class

# Calculate ROC-AUC score for model (one-vs-all approach for multi-class)
roc_auc = []

# Create a figure and axis
plt.figure(figsize=(8, 6))

# Plot ROC curve for each class
for i in range(num_classes):
    fpr, tpr, _ = roc_curve(y_test== i, y_pred_prob[:, i])
    roc_auc_class = auc(fpr, tpr)
    roc_auc.append(roc_auc_class * 100.0)
    plt.plot(fpr, tpr, label=f'Class {i} (AUC = {roc_auc_class:.2f})')

# Set plot title and labels
plt.title("Model ROC Curves for Multi-class Classification")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

## Converting y_train and y_test to one_hot

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical


label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train_original)
y_test_encoded = label_encoder.transform(y_test_original)

# Convert integer labels to one-hot encoding
y_train_one_hot = to_categorical(y_train_encoded)
y_test_one_hot = to_categorical(y_test_encoded)

# Check the shapes
print(y_train_one_hot.shape, y_test_one_hot.shape)

## Model 8 Stacked LSTM

In [ ]:
# Output Directory name:
output_dir = 'model_output/Stacked-LSTM' # Store model's parameter after each epoch, useful at a later time

# If the output_dir directory doesn't exist, we use the mkdirs() method to make it
if not os.path.exists(output_dir):
  os.makedirs(output_dir)

# Allow to save our model parameters after each epoch during training
modelcheckpoint = ModelCheckpoint(filepath = output_dir + '/weights.{epoch:02d}.hdf5')

In [ ]:
model8 = Sequential()
model8.add(Embedding(n_unique_words, n_dim, input_length=max_message_length))
model8.add(LSTM(100, return_sequences=True))
model8.add(Dropout(dropout))
model8.add(LSTM(100))
model8.add(Dropout(dropout))
model8.add(Dense(num_classes, activation='softmax'))  # Adjust the activation function
model8.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model8.summary()

start_time = time.time()
history8 = model8.fit(x_train, y_train_one_hot, validation_data=(x_test, y_test_one_hot), epochs=epochs, batch_size=batch_size, callbacks=[modelcheckpoint])
print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed

### Evaluate

In [ ]:
plt.figure(figsize=(10, 4))

# Visualizing training and validation accuracy for this Model
plt.subplot(1, 2, 1)
plt.plot(history8.history['accuracy'])
plt.plot(history8.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Visualizing training and validation loss for this Model
plt.subplot(1, 2, 2)
plt.plot(history8.history['loss'])
plt.plot(history8.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss/Error')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np

# Evaluating the model
y_pred_prob = model8.predict(x_test)
y_pred = y_pred_prob.argmax(axis=-1)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {100*accuracy:.2f}")

In [ ]:
# Plot ROC curve for model
plt.figure(figsize=(8, 6))
plt.hist(y_pred_prob)
_ = plt.axvline(x=0.5, color='orange')
plt.title("Model ROC-AUC")
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

# Assuming y_test_encoded and y_pred_prob are defined
# y_test_encoded: True labels (encoded, not one-hot encoded)
# y_pred_prob: Predicted probabilities for each class

# Calculate ROC-AUC score for model (one-vs-all approach for multi-class)
roc_auc = []

# Create a figure and axis
plt.figure(figsize=(8, 6))

# Plot ROC curve for each class
for i in range(num_classes):
    fpr, tpr, _ = roc_curve(y_test== i, y_pred_prob[:, i])
    roc_auc_class = auc(fpr, tpr)
    roc_auc.append(roc_auc_class * 100.0)
    plt.plot(fpr, tpr, label=f'Class {i} (AUC = {roc_auc_class:.2f})')

# Set plot title and labels
plt.title("Model ROC Curves for Multi-class Classification")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

## Model 9 Conv-LSTM

In [ ]:
# Output Directory name:
output_dir = 'model_output/CNN-LSTM' # Store model's parameter after each epoch, useful at a later time

# If the output_dir directory doesn't exist, we use the mkdirs() method to make it
if not os.path.exists(output_dir):
  os.makedirs(output_dir)

# Allow to save our model parameters after each epoch during training
modelcheckpoint = ModelCheckpoint(filepath = output_dir + '/weights.{epoch:02d}.hdf5')

In [ ]:
model9 = Sequential()
model9.add(Embedding(n_unique_words, n_dim, input_length=max_message_length))
model9.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model9.add(MaxPooling1D(pool_size=2))
model9.add(LSTM(100))
model9.add(Dropout(dropout))
model9.add(Dense(num_classes, activation='softmax'))  # Adjust the activation function
model9.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model9.summary()

start_time = time.time()
history9 = model9.fit(x_train, y_train_one_hot, validation_data=(x_test, y_test_one_hot), epochs=epochs, batch_size=batch_size, callbacks=[modelcheckpoint])
print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed

### Evaluate

In [ ]:
plt.figure(figsize=(10, 4))

# Visualizing training and validation accuracy for this Model
plt.subplot(1, 2, 1)
plt.plot(history9.history['accuracy'])
plt.plot(history9.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Visualizing training and validation loss for this Model
plt.subplot(1, 2, 2)
plt.plot(history9.history['loss'])
plt.plot(history9.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss/Error')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np

# Evaluating the model
y_pred_prob = model9.predict(x_test)
y_pred = y_pred_prob.argmax(axis=-1)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {100*accuracy:.2f}")

In [ ]:
# Plot ROC curve for model
plt.figure(figsize=(8, 6))
plt.hist(y_pred_prob)
_ = plt.axvline(x=0.5, color='orange')
plt.title("Model ROC-AUC")
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

# Assuming y_test_encoded and y_pred_prob are defined
# y_test_encoded: True labels (encoded, not one-hot encoded)
# y_pred_prob: Predicted probabilities for each class

# Calculate ROC-AUC score for model (one-vs-all approach for multi-class)
roc_auc = []

# Create a figure and axis
plt.figure(figsize=(8, 6))

# Plot ROC curve for each class
for i in range(num_classes):
    fpr, tpr, _ = roc_curve(y_test== i, y_pred_prob[:, i])
    roc_auc_class = auc(fpr, tpr)
    roc_auc.append(roc_auc_class * 100.0)
    plt.plot(fpr, tpr, label=f'Class {i} (AUC = {roc_auc_class:.2f})')

# Set plot title and labels
plt.title("Model ROC Curves for Multi-class Classification")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

## Model 10 Multi-ConvNet

In [ ]:
# Output Directory name:
output_dir = 'model_output/Multi-CovNet' # Store model's parameter after each epoch, useful at a later time

# If the output_dir directory doesn't exist, we use the mkdirs() method to make it
if not os.path.exists(output_dir):
  os.makedirs(output_dir)

# Allow to save our model parameters after each epoch during training
modelcheckpoint = ModelCheckpoint(filepath = output_dir + '/weights.{epoch:02d}.hdf5')

In [ ]:
model10 = Sequential()
model10.add(Embedding(n_unique_words, n_dim, input_length=max_message_length))
model10.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model10.add(GlobalMaxPooling1D())
model10.add(Dropout(dropout))
model10.add(Dense(512, activation='relu'))
model10.add(Dropout(dropout))
model10.add(Dense(num_classes, activation='softmax'))
model10.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model10.summary()

start_time = time.time()
history10 = model10.fit(x_train, y_train_one_hot, validation_data=(x_test, y_test_one_hot), epochs=epochs, batch_size=batch_size, callbacks=[modelcheckpoint])
print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed

### Evaluate

In [ ]:
plt.figure(figsize=(10, 4))

# Visualizing training and validation accuracy for this Model
plt.subplot(1, 2, 1)
plt.plot(history10.history['accuracy'])
plt.plot(history10.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Visualizing training and validation loss for this Model
plt.subplot(1, 2, 2)
plt.plot(history10.history['loss'])
plt.plot(history10.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss/Error')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np

# Evaluating the model
y_pred_prob = model10.predict(x_test)
y_pred = y_pred_prob.argmax(axis=-1)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {100*accuracy:.2f}")

In [ ]:
# Plot ROC curve for model
plt.figure(figsize=(8, 6))
plt.hist(y_pred_prob)
_ = plt.axvline(x=0.5, color='orange')
plt.title("Model ROC-AUC")
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

# Assuming y_test_encoded and y_pred_prob are defined
# y_test_encoded: True labels (encoded, not one-hot encoded)
# y_pred_prob: Predicted probabilities for each class

# Calculate ROC-AUC score for model (one-vs-all approach for multi-class)
roc_auc = []

# Create a figure and axis
plt.figure(figsize=(8, 6))

# Plot ROC curve for each class
for i in range(num_classes):
    fpr, tpr, _ = roc_curve(y_test== i, y_pred_prob[:, i])
    roc_auc_class = auc(fpr, tpr)
    roc_auc.append(roc_auc_class * 100.0)
    plt.plot(fpr, tpr, label=f'Class {i} (AUC = {roc_auc_class:.2f})')

# Set plot title and labels
plt.title("Model ROC Curves for Multi-class Classification")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

## Model 11 CNN-LSTM

In [ ]:
# Output Directory name:
output_dir = 'model_output/CNN-LSTM Part 2' # Store model's parameter after each epoch, useful at a later time

# If the output_dir directory doesn't exist, we use the mkdirs() method to make it
if not os.path.exists(output_dir):
  os.makedirs(output_dir)

# Allow to save our model parameters after each epoch during training
modelcheckpoint = ModelCheckpoint(filepath = output_dir + '/weights.{epoch:02d}.hdf5')

In [ ]:
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding
from keras.layers import LSTM
from keras.layers import SpatialDropout1D, Conv1D, GlobalMaxPooling1D # new!
from keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
# output directory name:
output_dir = 'model_output/conv'

drop_embed = 0.2 # new!
# convolutional layer architecture:
#n_conv = 256 # filters, a.k.a. kernels
#k_conv = 3 # kernel length
# dense layer architecture:
n_lstm = 256
drop_lstm = 0.2

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, SpatialDropout1D, LSTM, Dense

model11 = Sequential()
model11.add(Embedding(n_unique_words, n_dim, input_length=max_message_length))
model11.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model11.add(MaxPooling1D(pool_size=2))
model11.add(LSTM(100))
model11.add(Dropout(dropout))
model11.add(Dense(num_classes, activation='softmax'))
model11.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the summary of the model
model11.summary()

# Fitting the model
start_time = time.time()
history11 = model11.fit(x_train, y_train_one_hot, validation_data=(x_test, y_test_one_hot), epochs=epochs, batch_size=batch_size, callbacks=[modelcheckpoint])
print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed

### Evaluate

In [ ]:
plt.figure(figsize=(10, 4))

# Visualizing training and validation accuracy for this Model
plt.subplot(1, 2, 1)
plt.plot(history11.history['accuracy'])
plt.plot(history11.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Visualizing training and validation loss for this Model
plt.subplot(1, 2, 2)
plt.plot(history11.history['loss'])
plt.plot(history11.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss/Error')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np

# Evaluating the model
y_pred_prob = model11.predict(x_test)
y_pred = y_pred_prob.argmax(axis=-1)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {100*accuracy:.2f}")

In [ ]:
# Plot ROC curve for model
plt.figure(figsize=(8, 6))
plt.hist(y_pred_prob)
_ = plt.axvline(x=0.5, color='orange')
plt.title("Model ROC-AUC")
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

# Assuming y_test_encoded and y_pred_prob are defined
# y_test_encoded: True labels (encoded, not one-hot encoded)
# y_pred_prob: Predicted probabilities for each class

# Calculate ROC-AUC score for model (one-vs-all approach for multi-class)
roc_auc = []

# Create a figure and axis
plt.figure(figsize=(8, 6))

# Plot ROC curve for each class
for i in range(num_classes):
    fpr, tpr, _ = roc_curve(y_test== i, y_pred_prob[:, i])
    roc_auc_class = auc(fpr, tpr)
    roc_auc.append(roc_auc_class * 100.0)
    plt.plot(fpr, tpr, label=f'Class {i} (AUC = {roc_auc_class:.2f})')

# Set plot title and labels
plt.title("Model ROC Curves for Multi-class Classification")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

## Model 12 Dilated Convolutional Neural Network (DCNN)

In [ ]:
# Output Directory name:
output_dir = 'model_output/DCNN' # Store model's parameter after each epoch, useful at a later time

# If the output_dir directory doesn't exist, we use the mkdirs() method to make it
if not os.path.exists(output_dir):
  os.makedirs(output_dir)

# Allow to save our model parameters after each epoch during training
modelcheckpoint = ModelCheckpoint(filepath = output_dir + '/weights.{epoch:02d}.hdf5')

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dropout, Dense

model12 = Sequential()
model12.add(Embedding(n_unique_words, n_dim, input_length=max_message_length))
model12.add(Conv1D(filters=64, kernel_size=3, dilation_rate=2, activation='relu'))
model12.add(GlobalMaxPooling1D())
model12.add(Dense(128, activation='relu'))
model12.add(Dropout(0.5))
model12.add(Dense(num_classes, activation='softmax'))

model12.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model12.summary()

start_time = time.time()
history12 = model12.fit(x_train, y_train_one_hot, validation_data=(x_test, y_test_one_hot), epochs=epochs, batch_size=batch_size, callbacks=[modelcheckpoint])
print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed

### Evaluate

In [ ]:
plt.figure(figsize=(10, 4))

# Visualizing training and validation accuracy for this Model
plt.subplot(1, 2, 1)
plt.plot(history12.history['accuracy'])
plt.plot(history12.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Visualizing training and validation loss for this Model
plt.subplot(1, 2, 2)
plt.plot(history12.history['loss'])
plt.plot(history12.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss/Error')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np

# Evaluating the model
y_pred_prob = model12.predict(x_test)
y_pred = y_pred_prob.argmax(axis=-1)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {100*accuracy:.2f}")

In [ ]:
# Plot ROC curve for model
plt.figure(figsize=(8, 6))
plt.hist(y_pred_prob)
_ = plt.axvline(x=0.5, color='orange')
plt.title("Model ROC-AUC")
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

# Assuming y_test_encoded and y_pred_prob are defined
# y_test_encoded: True labels (encoded, not one-hot encoded)
# y_pred_prob: Predicted probabilities for each class

# Calculate ROC-AUC score for model (one-vs-all approach for multi-class)
roc_auc = []

# Create a figure and axis
plt.figure(figsize=(8, 6))

# Plot ROC curve for each class
for i in range(num_classes):
    fpr, tpr, _ = roc_curve(y_test== i, y_pred_prob[:, i])
    roc_auc_class = auc(fpr, tpr)
    roc_auc.append(roc_auc_class * 100.0)
    plt.plot(fpr, tpr, label=f'Class {i} (AUC = {roc_auc_class:.2f})')

# Set plot title and labels
plt.title("Model ROC Curves for Multi-class Classification")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

## Model 13 Residual Network (ResNet)

In [ ]:
# Output Directory name:
output_dir = 'model_output/Resnet' # Store model's parameter after each epoch, useful at a later time

# If the output_dir directory doesn't exist, we use the mkdirs() method to make it
if not os.path.exists(output_dir):
  os.makedirs(output_dir)

# Allow to save our model parameters after each epoch during training
modelcheckpoint = ModelCheckpoint(filepath = output_dir + '/weights.{epoch:02d}.hdf5')

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalAveragePooling1D, Dense

model13 = Sequential()
model13.add(Embedding(n_unique_words, n_dim, input_length=max_message_length))
model13.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'))
model13.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'))
model13.add(MaxPooling1D(pool_size=2))
model13.add(Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
model13.add(Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
model13.add(MaxPooling1D(pool_size=2))
model13.add(GlobalAveragePooling1D())
model13.add(Dense(128, activation='relu'))
model13.add(Dense(num_classes, activation='softmax'))

model13.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model13.summary()

start_time = time.time()
history13 = model13.fit(x_train, y_train_one_hot, validation_data=(x_test, y_test_one_hot), epochs=epochs, batch_size=batch_size, callbacks=[modelcheckpoint])
print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed

### Evaluate

In [ ]:
plt.figure(figsize=(10, 4))

# Visualizing training and validation accuracy for this Model
plt.subplot(1, 2, 1)
plt.plot(history13.history['accuracy'])
plt.plot(history13.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Visualizing training and validation loss for this Model
plt.subplot(1, 2, 2)
plt.plot(history13.history['loss'])
plt.plot(history13.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss/Error')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np

# Evaluating the model
y_pred_prob = model13.predict(x_test)
y_pred = y_pred_prob.argmax(axis=-1)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {100*accuracy:.2f}")

In [ ]:
# Plot ROC curve for model
plt.figure(figsize=(8, 6))
plt.hist(y_pred_prob)
_ = plt.axvline(x=0.5, color='orange')
plt.title("Model ROC-AUC")
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

# Assuming y_test_encoded and y_pred_prob are defined
# y_test_encoded: True labels (encoded, not one-hot encoded)
# y_pred_prob: Predicted probabilities for each class

# Calculate ROC-AUC score for model (one-vs-all approach for multi-class)
roc_auc = []

# Create a figure and axis
plt.figure(figsize=(8, 6))

# Plot ROC curve for each class
for i in range(num_classes):
    fpr, tpr, _ = roc_curve(y_test== i, y_pred_prob[:, i])
    roc_auc_class = auc(fpr, tpr)
    roc_auc.append(roc_auc_class * 100.0)
    plt.plot(fpr, tpr, label=f'Class {i} (AUC = {roc_auc_class:.2f})')

# Set plot title and labels
plt.title("Model ROC Curves for Multi-class Classification")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

## Model 14 Variational autoencoder (VAE) Network

In [ ]:
# Output Directory name:
output_dir = 'model_output/VAE' # Store model's parameter after each epoch, useful at a later time

# If the output_dir directory doesn't exist, we use the mkdirs() method to make it
if not os.path.exists(output_dir):
  os.makedirs(output_dir)

# Allow to save our model parameters after each epoch during training
modelcheckpoint = ModelCheckpoint(filepath = output_dir + '/weights.{epoch:02d}.hdf5')

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense, Lambda, Input
from keras import backend as K
from keras.models import Model

# Define the input layer
input_layer = Input(shape=(max_message_length,))

# Embedding layer
embedding_layer = Embedding(input_dim=n_unique_words, output_dim=n_dim, input_length=max_message_length)(input_layer)

# Flatten the embedding output
flatten_layer = Flatten()(embedding_layer)

# Intermediate dense layers
hidden_layer_1 = Dense(128, activation='relu')(flatten_layer)
hidden_layer_2 = Dense(64, activation='relu')(hidden_layer_1)

# Output layers for mean and log variance
z_mean = Dense(32, name='z_mean')(hidden_layer_2)
z_log_var = Dense(32, name='z_log_var')(hidden_layer_2)

# Sampling function
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

# Create a Lambda layer for the sampling function
z = Lambda(sampling, output_shape=(32,), name='z')([z_mean, z_log_var])

# Dense layer with softmax activation for multiclass classification
output_layer = Dense(num_classes, activation='softmax', name='output')(z)

# Create a new model with input and output layers
model14 = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model14.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model14.summary()

start_time = time.time()
history14 = model14.fit(x_train, y_train_one_hot, validation_data=(x_test, y_test_one_hot), epochs=epochs, batch_size=batch_size, callbacks=[modelcheckpoint])
print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed

### Evaluate

In [ ]:
plt.figure(figsize=(10, 4))

# Visualizing training and validation accuracy for this Model
plt.subplot(1, 2, 1)
plt.plot(history14.history['accuracy'])
plt.plot(history14.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Visualizing training and validation loss for this Model
plt.subplot(1, 2, 2)
plt.plot(history14.history['loss'])
plt.plot(history14.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss/Error')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np

# Evaluating the model
y_pred_prob = model14.predict(x_test)
y_pred = y_pred_prob.argmax(axis=-1)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {100*accuracy:.2f}")

In [ ]:
# Plot ROC curve for model
plt.figure(figsize=(8, 6))
plt.hist(y_pred_prob)
_ = plt.axvline(x=0.5, color='orange')
plt.title("Model ROC-AUC")
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

# Assuming y_test_encoded and y_pred_prob are defined
# y_test_encoded: True labels (encoded, not one-hot encoded)
# y_pred_prob: Predicted probabilities for each class

# Calculate ROC-AUC score for model (one-vs-all approach for multi-class)
roc_auc = []

# Create a figure and axis
plt.figure(figsize=(8, 6))

# Plot ROC curve for each class
for i in range(num_classes):
    fpr, tpr, _ = roc_curve(y_test== i, y_pred_prob[:, i])
    roc_auc_class = auc(fpr, tpr)
    roc_auc.append(roc_auc_class * 100.0)
    plt.plot(fpr, tpr, label=f'Class {i} (AUC = {roc_auc_class:.2f})')

# Set plot title and labels
plt.title("Model ROC Curves for Multi-class Classification")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

## Model 15 Bi-GRU

In [ ]:
# Output Directory name:
output_dir = 'model_output/Bi-GRU' # Store model's parameter after each epoch, useful at a later time

# If the output_dir directory doesn't exist, we use the mkdirs() method to make it
if not os.path.exists(output_dir):
  os.makedirs(output_dir)

# Allow to save our model parameters after each epoch during training
modelcheckpoint = ModelCheckpoint(filepath = output_dir + '/weights.{epoch:02d}.hdf5')

In [ ]:
from keras.layers import Bidirectional, GRU, Dense

model15 = Sequential()
model15.add(Embedding(n_unique_words, n_dim, input_length=max_message_length))
model15.add(Bidirectional(GRU(64)))
model15.add(Dense(64, activation='relu'))
model15.add(Dense(num_classes, activation='softmax'))
model15.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model15.summary()

start_time = time.time()
history15 = model15.fit(x_train, y_train_one_hot, validation_data=(x_test, y_test_one_hot), epochs=epochs, batch_size=batch_size, callbacks=[modelcheckpoint])
print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed

### Evaluate

In [ ]:
plt.figure(figsize=(10, 4))

# Visualizing training and validation accuracy for this Model
plt.subplot(1, 2, 1)
plt.plot(history15.history['accuracy'])
plt.plot(history15.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Visualizing training and validation loss for this Model
plt.subplot(1, 2, 2)
plt.plot(history15.history['loss'])
plt.plot(history15.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss/Error')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np

# Evaluating the model
y_pred_prob = model15.predict(x_test)
y_pred = y_pred_prob.argmax(axis=-1)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {100*accuracy:.2f}")

In [ ]:
# Plot ROC curve for model
plt.figure(figsize=(8, 6))
plt.hist(y_pred_prob)
_ = plt.axvline(x=0.5, color='orange')
plt.title("Model ROC-AUC")
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

# Assuming y_test_encoded and y_pred_prob are defined
# y_test_encoded: True labels (encoded, not one-hot encoded)
# y_pred_prob: Predicted probabilities for each class

# Calculate ROC-AUC score for model (one-vs-all approach for multi-class)
roc_auc = []

# Create a figure and axis
plt.figure(figsize=(8, 6))

# Plot ROC curve for each class
for i in range(num_classes):
    fpr, tpr, _ = roc_curve(y_test== i, y_pred_prob[:, i])
    roc_auc_class = auc(fpr, tpr)
    roc_auc.append(roc_auc_class * 100.0)
    plt.plot(fpr, tpr, label=f'Class {i} (AUC = {roc_auc_class:.2f})')

# Set plot title and labels
plt.title("Model ROC Curves for Multi-class Classification")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

## Model 16 Multi-Layer Perceptron (MLP)

In [ ]:
# Output Directory name:
output_dir = 'model_output/MLP' # Store model's parameter after each epoch, useful at a later time

# If the output_dir directory doesn't exist, we use the mkdirs() method to make it
if not os.path.exists(output_dir):
  os.makedirs(output_dir)

# Allow to save our model parameters after each epoch during training
modelcheckpoint = ModelCheckpoint(filepath = output_dir + '/weights.{epoch:02d}.hdf5')

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten

model16 = Sequential()
model16.add(Dense(512, activation='relu', input_shape=(100,)))
model16.add(Dense(256, activation='relu'))
model16.add(Dense(128, activation='relu'))
model16.add(Dense(64, activation='relu'))
model16.add(Dense(32, activation='relu'))
model16.add(Dense(num_classes, activation='softmax'))
model16.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model16.summary()

start_time = time.time()
history16 = model16.fit(x_train, y_train_one_hot, validation_data=(x_test, y_test_one_hot), epochs=epochs, batch_size=batch_size, callbacks=[modelcheckpoint])
print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed

### Evaluate

In [ ]:
plt.figure(figsize=(10, 4))

# Visualizing training and validation accuracy for this Model
plt.subplot(1, 2, 1)
plt.plot(history16.history['accuracy'])
plt.plot(history16.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Visualizing training and validation loss for this Model
plt.subplot(1, 2, 2)
plt.plot(history16.history['loss'])
plt.plot(history16.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss/Error')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np

# Evaluating the model
y_pred_prob = model16.predict(x_test)
y_pred = y_pred_prob.argmax(axis=-1)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {100*accuracy:.2f}")

In [ ]:
# Plot ROC curve for model
plt.figure(figsize=(8, 6))
plt.hist(y_pred_prob)
_ = plt.axvline(x=0.5, color='orange')
plt.title("Model ROC-AUC")
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

# Assuming y_test_encoded and y_pred_prob are defined
# y_test_encoded: True labels (encoded, not one-hot encoded)
# y_pred_prob: Predicted probabilities for each class

# Calculate ROC-AUC score for model (one-vs-all approach for multi-class)
roc_auc = []

# Create a figure and axis
plt.figure(figsize=(8, 6))

# Plot ROC curve for each class
for i in range(num_classes):
    fpr, tpr, _ = roc_curve(y_test== i, y_pred_prob[:, i])
    roc_auc_class = auc(fpr, tpr)
    roc_auc.append(roc_auc_class * 100.0)
    plt.plot(fpr, tpr, label=f'Class {i} (AUC = {roc_auc_class:.2f})')

# Set plot title and labels
plt.title("Model ROC Curves for Multi-class Classification")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

## Model 17 CNN-GRU

In [ ]:
# Output Directory name:
output_dir = 'model_output/CNN-GRU' # Store model's parameter after each epoch, useful at a later time

# If the output_dir directory doesn't exist, we use the mkdirs() method to make it
if not os.path.exists(output_dir):
  os.makedirs(output_dir)

# Allow to save our model parameters after each epoch during training
modelcheckpoint = ModelCheckpoint(filepath = output_dir + '/weights.{epoch:02d}.hdf5')

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, GRU, Dense, Dropout, Flatten


model17 = Sequential()

# Embedding layer
model17.add(Embedding(n_unique_words, n_dim, input_length=max_message_length))

# Convolutional layers
model17.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model17.add(MaxPooling1D(pool_size=2))

# GRU layer
model17.add(GRU(100, dropout=dropout, recurrent_dropout=dropout))

# Fully connected layers
model17.add(Dense(64, activation='relu'))
model17.add(Dropout(dropout))

# Output layer
model17.add(Dense(num_classes, activation='softmax'))

# Compile the model
model17.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the summary of the model
model17.summary()

start_time = time.time()
history17 = model17.fit(x_train, y_train_one_hot, validation_data=(x_test, y_test_one_hot), epochs=epochs, batch_size=batch_size, callbacks=[modelcheckpoint])
print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed

In [ ]:
# Evaluate the model on the test set
loss17, accuracy17 = model17.evaluate(x_test, y_test_one_hot)
print(f'Test Loss: {loss17}, Test Accuracy: {100*accuracy17:.2f}')

### Evaluate

In [ ]:
plt.figure(figsize=(10, 4))

# Visualizing training and validation accuracy for this Model
plt.subplot(1, 2, 1)
plt.plot(history17.history['accuracy'])
plt.plot(history17.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Visualizing training and validation loss for this Model
plt.subplot(1, 2, 2)
plt.plot(history17.history['loss'])
plt.plot(history17.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss/Error')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np

# Evaluating the model
y_pred_prob = model17.predict(x_test)
y_pred = y_pred_prob.argmax(axis=-1)


# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {100*accuracy:.2f}")

In [ ]:
# Plot ROC curve for model
plt.figure(figsize=(8, 6))
plt.hist(y_pred_prob)
_ = plt.axvline(x=0.5, color='orange')
plt.title("Model ROC-AUC")
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

# Assuming y_test_encoded and y_pred_prob are defined
# y_test_encoded: True labels (encoded, not one-hot encoded)
# y_pred_prob: Predicted probabilities for each class

# Calculate ROC-AUC score for model (one-vs-all approach for multi-class)
roc_auc = []

# Create a figure and axis
plt.figure(figsize=(8, 6))

# Plot ROC curve for each class
for i in range(num_classes):
    fpr, tpr, _ = roc_curve(y_test== i, y_pred_prob[:, i])
    roc_auc_class = auc(fpr, tpr)
    roc_auc.append(roc_auc_class * 100.0)
    plt.plot(fpr, tpr, label=f'Class {i} (AUC = {roc_auc_class:.2f})')

# Set plot title and labels
plt.title("Model ROC Curves for Multi-class Classification")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

## Model 18 CNN + Bi-GRU

In [ ]:
# Output Directory name:
output_dir = 'model_output/CNN-Bi-GRU' # Store model's parameter after each epoch, useful at a later time

# If the output_dir directory doesn't exist, we use the mkdirs() method to make it
if not os.path.exists(output_dir):
  os.makedirs(output_dir)

# Allow to save our model parameters after each epoch during training
modelcheckpoint = ModelCheckpoint(filepath = output_dir + '/weights.{epoch:02d}.hdf5')

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Bidirectional, GRU, Dense, Dropout, Flatten


model18 = Sequential()

# Embedding layer
model18.add(Embedding(n_unique_words, n_dim, input_length=max_message_length))

# Convolutional layers
model18.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model18.add(MaxPooling1D(pool_size=2))

# Bidirectional GRU layer
model18.add(Bidirectional(GRU(100, dropout=dropout, recurrent_dropout=dropout)))

# Fully connected layers
model18.add(Dense(64, activation='relu'))
model18.add(Dropout(dropout))

# Output layer
model18.add(Dense(num_classes, activation='softmax'))

# Compile the model
model18.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the summary of the model
model18.summary()

start_time = time.time()
history18 = model18.fit(x_train, y_train_one_hot, validation_data=(x_test, y_test_one_hot), epochs=epochs, batch_size=batch_size, callbacks=[modelcheckpoint])
print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed

In [ ]:
# Evaluate the model on the test set
loss18, accuracy18 = model18.evaluate(x_test, y_test_one_hot)
print(f'Test Loss: {loss18}, Test Accuracy: {100*accuracy18:.2f}')

### Evaluate

In [ ]:
plt.figure(figsize=(10, 4))

# Visualizing training and validation accuracy for this Model
plt.subplot(1, 2, 1)
plt.plot(history18.history['accuracy'])
plt.plot(history18.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Visualizing training and validation loss for this Model
plt.subplot(1, 2, 2)
plt.plot(history18.history['loss'])
plt.plot(history18.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss/Error')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np

# Evaluating the model
y_pred_prob = model18.predict(x_test)
y_pred = y_pred_prob.argmax(axis=-1)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {100*accuracy:.2f}")

In [ ]:
# Plot ROC curve for model
plt.figure(figsize=(8, 6))
plt.hist(y_pred_prob)
_ = plt.axvline(x=0.5, color='orange')
plt.title("Model ROC-AUC")
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

# Assuming y_test_encoded and y_pred_prob are defined
# y_test_encoded: True labels (encoded, not one-hot encoded)
# y_pred_prob: Predicted probabilities for each class

# Calculate ROC-AUC score for model (one-vs-all approach for multi-class)
roc_auc = []

# Create a figure and axis
plt.figure(figsize=(8, 6))

# Plot ROC curve for each class
for i in range(num_classes):
    fpr, tpr, _ = roc_curve(y_test== i, y_pred_prob[:, i])
    roc_auc_class = auc(fpr, tpr)
    roc_auc.append(roc_auc_class * 100.0)
    plt.plot(fpr, tpr, label=f'Class {i} (AUC = {roc_auc_class:.2f})')

# Set plot title and labels
plt.title("Model ROC Curves for Multi-class Classification")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

## Model 19 CNN + Bi-LSTM

In [ ]:
# Output Directory name:
output_dir = 'model_output/CNN-Bi-LSTM' # Store model's parameter after each epoch, useful at a later time

# If the output_dir directory doesn't exist, we use the mkdirs() method to make it
if not os.path.exists(output_dir):
  os.makedirs(output_dir)

# Allow to save our model parameters after each epoch during training
modelcheckpoint = ModelCheckpoint(filepath = output_dir + '/weights.{epoch:02d}.hdf5')

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense, Dropout, Flatten


model19 = Sequential()

# Embedding layer
model19.add(Embedding(n_unique_words, n_dim, input_length=max_message_length))

# Convolutional layers
model19.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model19.add(MaxPooling1D(pool_size=2))

# Bidirectional LSTM layer
model19.add(Bidirectional(LSTM(100, dropout=dropout, recurrent_dropout=dropout)))

# Fully connected layers
model19.add(Dense(64, activation='relu'))
model19.add(Dropout(dropout))

# Output layer
model19.add(Dense(num_classes, activation='softmax'))

# Compile the model
model19.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the summary of the model
model19.summary()

start_time = time.time()
history19 = model19.fit(x_train, y_train_one_hot, validation_data=(x_test, y_test_one_hot), epochs=epochs, batch_size=batch_size, callbacks=[modelcheckpoint])
print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed

In [ ]:
# Evaluate the model on the test set
loss19, accuracy19 = model19.evaluate(x_test, y_test_one_hot)
print(f'Test Loss: {loss19}, Test Accuracy: {100*accuracy19:.2f}')

### Evaluate

In [ ]:
plt.figure(figsize=(10, 4))

# Visualizing training and validation accuracy for this Model
plt.subplot(1, 2, 1)
plt.plot(history19.history['accuracy'])
plt.plot(history19.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Visualizing training and validation loss for this Model
plt.subplot(1, 2, 2)
plt.plot(history19.history['loss'])
plt.plot(history19.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss/Error')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np

# Evaluating the model
y_pred_prob = model19.predict(x_test)
y_pred = y_pred_prob.argmax(axis=-1)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {100*accuracy:.2f}")

In [ ]:
# Plot ROC curve for model
plt.figure(figsize=(8, 6))
plt.hist(y_pred_prob)
_ = plt.axvline(x=0.5, color='orange')
plt.title("Model ROC-AUC")
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

# Assuming y_test_encoded and y_pred_prob are defined
# y_test_encoded: True labels (encoded, not one-hot encoded)
# y_pred_prob: Predicted probabilities for each class

# Calculate ROC-AUC score for model (one-vs-all approach for multi-class)
roc_auc = []

# Create a figure and axis
plt.figure(figsize=(8, 6))

# Plot ROC curve for each class
for i in range(num_classes):
    fpr, tpr, _ = roc_curve(y_test== i, y_pred_prob[:, i])
    roc_auc_class = auc(fpr, tpr)
    roc_auc.append(roc_auc_class * 100.0)
    plt.plot(fpr, tpr, label=f'Class {i} (AUC = {roc_auc_class:.2f})')

# Set plot title and labels
plt.title("Model ROC Curves for Multi-class Classification")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

## Model 20 BERT Model

In [ ]:
# Output Directory name:
output_dir = 'model_output/BERT' # Store model's parameter after each epoch, useful at a later time

# If the output_dir directory doesn't exist, we use the mkdirs() method to make it
if not os.path.exists(output_dir):
  os.makedirs(output_dir)

# Allow to save our model parameters after each epoch during training
modelcheckpoint = ModelCheckpoint(filepath = output_dir + '/weights.{epoch:02d}.hdf5')

In [ ]:
# Split the data into training and testing sets
X_train_for_tokens, X_val_for_tokens, y_train_for_tokens, y_val_for_tokens = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the training data
train_tokens = tokenizer(
    X_train_for_tokens.tolist(),
    truncation=True,
    padding=True,
    max_length=128,  # Adjust as needed
    return_tensors='tf',  # Use 'tf' for TensorFlow
)

# Tokenize the validation data
val_tokens = tokenizer(
    X_val_for_tokens.tolist(),
    truncation=True,
    padding=True,
    max_length=128,  # Adjust as needed
    return_tensors='tf',  # Use 'tf' for TensorFlow
)

# Display the tokenized input
print(train_tokens)
print(val_tokens)


In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from transformers import BertTokenizer, TFBertModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Load the BERT model with pooler layer set to non-trainable
bert_model = TFBertModel.from_pretrained('bert-base-uncased', trainable=False)

# Define the input layer
input_layer = Input(shape=(128,), dtype=tf.int32, name='input_ids')

# Connect the input layer to the BERT model
bert_output = bert_model(input_layer)[0]

# Global average pooling
pooled_output = tf.reduce_mean(bert_output, axis=1)

# Add a dense layer for classification
dense_layer = Dense(256, activation='relu')(pooled_output)

# Output layer for multiclass classification
output_layer = Dense(num_classes, activation='softmax')(dense_layer)

# Create the model
model_bert = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model_bert.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model_bert.summary()

In [ ]:
# Convert the labels to NumPy arrays and one-hot encode them
encoder = OneHotEncoder(sparse=False)
y_train_for_tokens_encoded = encoder.fit_transform(y_train_for_tokens.values.reshape(-1, 1))
y_val_for_tokens_encoded = encoder.transform(y_val_for_tokens.values.reshape(-1, 1))

In [ ]:
# Train the model
start_time = time.time()
history_model_bert = model_bert.fit(
    train_tokens['input_ids'],
    y_train_for_tokens_encoded,
    validation_data=(val_tokens['input_ids'], y_val_for_tokens_encoded),
    epochs=2,  # Adjust as needed
    batch_size=256,  # Adjust as needed
    callbacks=[modelcheckpoint]
)
print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed

In [ ]:
plt.figure(figsize=(10, 4))

# Visualizing training and validation accuracy for this Model
plt.subplot(1, 2, 1)
plt.plot(history_model_bert.history['accuracy'])
plt.plot(history_model_bert.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Visualizing training and validation loss for this Model
plt.subplot(1, 2, 2)
plt.plot(history_model_bert.history['loss'])
plt.plot(history_model_bert.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss/Error')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()